In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy

In [3]:
p = numpy.array(
    [
        [11, 12, 13],
        [21, 22, 23],
        [31, 32, 33],
    ]
)
p

array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

In [4]:
import lasp.utils
p1 = lasp.utils.circshift(p, shift=1-numpy.array([2, 2]))
print(p1)

[[22 23 21]
 [32 33 31]
 [12 13 11]]


In [5]:
def make_bccb(p: numpy.ndarray) -> numpy.ndarray :
    
    dim = numpy.prod(numpy.array(p.shape))
    p_cpy = numpy.copy(p)
    
    p1 = numpy.reshape(p_cpy.T, newshape=dim)
    P = numpy.zeros(shape=(dim, dim))
    P[:, 0] = p1

    for i in range(1, 3):
        for j in range(0, 3*3, 3):
            P[j:j+3, i] = numpy.roll(P[j:j+3, i-1], 1)

    for i in range(1, 3):
        P[:, 3*i: 3*i+3] = numpy.roll(P[:, 3*(i-1): 3*(i-1)+3], (3, 0), (0, 1))


    return P

In [6]:
bccb = make_bccb(p1)
print(bccb)

[[22. 12. 32. 21. 11. 31. 23. 13. 33.]
 [32. 22. 12. 31. 21. 11. 33. 23. 13.]
 [12. 32. 22. 11. 31. 21. 13. 33. 23.]
 [23. 13. 33. 22. 12. 32. 21. 11. 31.]
 [33. 23. 13. 32. 22. 12. 31. 21. 11.]
 [13. 33. 23. 12. 32. 22. 11. 31. 21.]
 [21. 11. 31. 23. 13. 33. 22. 12. 32.]
 [31. 21. 11. 33. 23. 13. 32. 22. 12.]
 [11. 31. 21. 13. 33. 23. 12. 32. 22.]]


In [7]:
from  scipy.fft import fft2, ifft2
import numpy.linalg

# H = F^{T} D F
# Hx = F^{T} D F x = ifft2(D *. fft2(x))

y = numpy.identity(9)
h = p1

n, m = h.shape
h_pad = numpy.zeros_like(y)
h_pad[0:n, 0:m] = numpy.copy(h)
center = numpy.array(h.shape) // 2

h_pad_shifted = lasp.utils.circshift(h_pad, 1-center)

# Eigens values of patrix bccb generate by h
bccb_eigen_values = fft2(h_pad_shifted)

print(bccb @ y)

numpy.real(ifft2(bccb_eigen_values * fft2(y)))






# print(eigen_values)
# print(scipy.fft.ifft2(eigen_values * numpy.fft.fft2(numpy.identity(numpy.min(eigen_values.shape)))))

[[22. 12. 32. 21. 11. 31. 23. 13. 33.]
 [32. 22. 12. 31. 21. 11. 33. 23. 13.]
 [12. 32. 22. 11. 31. 21. 13. 33. 23.]
 [23. 13. 33. 22. 12. 32. 21. 11. 31.]
 [33. 23. 13. 32. 22. 12. 31. 21. 11.]
 [13. 33. 23. 12. 32. 22. 11. 31. 21.]
 [21. 11. 31. 23. 13. 33. 22. 12. 32.]
 [31. 21. 11. 33. 23. 13. 32. 22. 12.]
 [11. 31. 21. 13. 33. 23. 12. 32. 22.]]


array([[ 6.60000000e+01,  5.40000000e+01,  2.10000000e+01,
        -2.30755522e-15,  8.88178420e-16, -1.06581410e-14,
         2.30755522e-15,  1.20000000e+01,  4.50000000e+01],
       [ 4.50000000e+01,  6.60000000e+01,  5.40000000e+01,
         2.10000000e+01,  4.32189875e-15,  8.88178420e-16,
        -3.55271368e-15,  2.78352860e-15,  1.20000000e+01],
       [ 1.20000000e+01,  4.50000000e+01,  6.60000000e+01,
         5.40000000e+01,  2.10000000e+01, -7.69185075e-16,
         0.00000000e+00,  3.55271368e-15,  7.69185075e-16],
       [ 0.00000000e+00,  1.20000000e+01,  4.50000000e+01,
         6.60000000e+01,  5.40000000e+01,  2.10000000e+01,
         0.00000000e+00,  0.00000000e+00, -3.55271368e-15],
       [-7.10542736e-15,  3.55271368e-15,  1.20000000e+01,
         4.50000000e+01,  6.60000000e+01,  5.40000000e+01,
         2.10000000e+01,  7.10542736e-15,  3.55271368e-15],
       [ 0.00000000e+00,  0.00000000e+00,  7.10542736e-15,
         1.20000000e+01,  4.50000000e+01,  6.600000

In [8]:
x = numpy.array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])
numpy.roll(x, (1, 0), (0, 1))
# array([[9, 0, 1, 2, 3],
#        [4, 5, 6, 7, 8]])

array([[5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4]])

In [9]:
def make_bccb(h: numpy.ndarray) -> numpy.ndarray :
    
    size_block = h.shape[0]

    center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
    shift = 1-center
    h_shift = lasp.utils.circshift(h, shift)

    dim = size_block**2

    col = numpy.reshape(h_shift.T, newshape=dim)
    
    bccb = numpy.zeros(shape=(dim, dim))
    bccb[:, 0] = col

    for i in range(1, h.shape[0]):
        for j in range(0, size_block**2, size_block):
            bccb[j:j+size_block, i] = numpy.roll(bccb[j:j+size_block, i-1], 1)

    for i in range(1, 3):
        bccb[:, size_block*i: size_block*i+size_block] = \
            numpy.roll(
                bccb[:, size_block*(i-1): size_block*(i-1)+size_block], 
                (size_block, 0), 
                (0, 1)
            )


    return bccb

In [10]:
h = numpy.array(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]
)

x_mat = numpy.reshape(numpy.arange(0, 9), (3, 3))
x_vec = numpy.reshape(x_mat, newshape=9, order='F')
print(x_mat)
print(x_vec)


[[0 1 2]
 [3 4 5]
 [6 7 8]]
[0 3 6 1 4 7 2 5 8]


In [11]:
H = make_bccb(h)
H @ x_vec

array([210., 129., 210., 201., 120., 201., 210., 129., 210.])

In [12]:
import scipy.fft

size_block = h.shape[0]
center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
shift = 1-center
h_shift = lasp.utils.circshift(h, shift)

s = scipy.fft.fft2(h_shift)
b = scipy.fft.ifft2(s * scipy.fft.fft(x_mat))
b = numpy.real(b)
print(b)


[[ 16.   13.   16. ]
 [ 65.5  62.5  65.5]
 [-33.5 -36.5 -33.5]]


In [13]:
import scipy.fft

lamda = scipy.fft.fft(H[:, 0])
lamda

array([45.         -0.j        , -2.81907786 -3.35964617j,
        2.29813333 +0.40522291j,  0.        -15.58845727j,
        0.52094453 -1.43128334j,  0.52094453 +1.43128334j,
        0.        +15.58845727j,  2.29813333 -0.40522291j,
       -2.81907786 +3.35964617j])

In [14]:
size_block = h.shape[0]

center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
shift = 1-center
h_shift = lasp.utils.circshift(h, shift)
scipy.fft.fft2(h_shift)

array([[45. -0.j        ,  0. -5.19615242j,  0. +5.19615242j],
       [ 0.-15.58845727j,  0. +0.j        ,  0. -0.j        ],
       [ 0.+15.58845727j,  0. +0.j        ,  0. -0.j        ]])

In [15]:
x_vec = numpy.arange(0, 9)
x_mat = numpy.reshape(x_vec, (3, 3))

In [16]:
print(x_vec)
print(x_mat)

[0 1 2 3 4 5 6 7 8]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [17]:
scipy.fft.fft2(x_mat)

array([[ 36. -0.j        ,  -4.5+2.59807621j,  -4.5-2.59807621j],
       [-13.5+7.79422863j,   0. +0.j        ,   0. -0.j        ],
       [-13.5-7.79422863j,   0. +0.j        ,   0. -0.j        ]])

In [18]:
numpy.round(1.5)

2.0